## Fine-tuning torchvision models

### Tutorial from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#inception-v3

In [2]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.13.1
Torchvision Version:  0.15.0a0+9b233d4


In [35]:
# Number of classes in the dataset
num_classes = 2
# Batch size for training (change depending on how much memory you have)
batch_size = 16
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

Define a few helper functions

In [30]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # phase = "train"
        # model.train()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # # Iterate over data.
            # for inputs, labels in dataloaders:
            #     inputs = inputs.to(device)
            #     labels = labels.to(device)
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # epoch_loss = running_loss / len(dataloaders.dataset)
            # epoch_acc = running_corrects.double() / len(dataloaders.dataset)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [5]:
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Need to reshape both the primary output layer and the auxiliary output layer
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 299

    return model_ft, input_size

In [6]:
# Initialize the model for this run
model_ft, input_size = initialize_model(num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

/Users/alexandrasmith/miniforge3/envs/masters/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alexandrasmith/miniforge3/envs/masters/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
# Load data
SAVE_DEST = '/Users/alexandrasmith/Desktop/Workspace/Projects/masters/data/processed/tests/'
PATCH_SIZE=1024
STRIDE=PATCH_SIZE

patches = torch.load(SAVE_DEST + "00-patches.pt")
labels = torch.load(SAVE_DEST + "00-labels.pt")
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model_input = preprocess(patches)



# trainloader = DataLoader(model_input, batch_size=32, shuffle=True, drop_last=True)
data = data_utils.TensorDataset(model_input, labels)
train_set, val_set = data_utils.random_split(data, [515, 222])
# dataloader = data_utils.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

train_loader = data_utils.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = data_utils.DataLoader(val_set, batch_size=batch_size, shuffle=True, drop_last=True)

In [37]:
# Create training and validation dataloaders
dataloaders_dict = {'train': train_loader, 'val': val_loader}
print(dataloaders_dict)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x157c5cd90>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x157afaaf0>}


In [38]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [41]:
# Observe that all parameters are being optimized
optimizer_ft = optim.RMSprop(params_to_update, lr=0.1, weight_decay=0.9, momentum=0.9, eps=1.0)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)

Epoch 0/14
----------
train Loss: 6.2658 Acc: 0.5437
val Loss: 0.4573 Acc: 0.7883

Epoch 1/14
----------
train Loss: 8.7676 Acc: 0.5534
val Loss: 3.0552 Acc: 0.3829

Epoch 2/14
----------
train Loss: 6.5809 Acc: 0.5165
val Loss: 5.8723 Acc: 0.3874

Epoch 3/14
----------
train Loss: 7.3325 Acc: 0.5515
val Loss: 6.3727 Acc: 0.3874

Epoch 4/14
----------
train Loss: 6.1727 Acc: 0.5379
val Loss: 6.5908 Acc: 0.3829

Epoch 5/14
----------
train Loss: 3.4675 Acc: 0.5728
val Loss: 3.0533 Acc: 0.5631

Epoch 6/14
----------
train Loss: 2.9590 Acc: 0.6117
val Loss: 2.1019 Acc: 0.4189

Epoch 7/14
----------
train Loss: 4.9292 Acc: 0.5786
val Loss: 0.4591 Acc: 0.7387

Epoch 8/14
----------
train Loss: 7.4369 Acc: 0.5515
val Loss: 4.1477 Acc: 0.3649

Epoch 9/14
----------
train Loss: 11.0690 Acc: 0.4718
val Loss: 17.8299 Acc: 0.3829

Epoch 10/14
----------
train Loss: 12.4964 Acc: 0.5029
val Loss: 2.6091 Acc: 0.4099

Epoch 11/14
----------
train Loss: 9.5399 Acc: 0.4796
val Loss: 3.4384 Acc: 0.5586


Look at data used in tutorial

In [13]:
data_dir = "/Users/alexandrasmith/Downloads/hymenoptera_data"

In [14]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

Initializing Datasets and Dataloaders...


In [33]:
tr = dataloaders_dict['train']
val = dataloaders_dict['val']

train_features, train_labels = next(iter(tr))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]

Feature batch shape: torch.Size([8, 3, 299, 299])
Labels batch shape: torch.Size([8])


In [34]:
print(len(tr.dataset))
print(len(val.dataset))

244
153


In [35]:
print(len(tr))

31
